In [3]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, RocCurveDisplay, roc_curve, auc

# Chicago Car Crash Analysis

## Overview 
add overview 

## Business Problem

add buisness problem

## Data Source 

Car crash and vehicle accident data was sourced from the [Chicago Data Portal](https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d/about_data). 

Data collected between January 1st, 2017 to September 26, 2024. 


## Results

add results

# Data Import and Cleaning 

In [4]:
df_vehicles = pd.read_csv("data/Traffic_Crashes_Vehicles.csv")
df_people = pd.read_csv("data/Traffic_Crashes_People.csv")
df_crashes = pd.read_csv("data/Traffic_Crashes_Crashes.csv")

/var/folders/4f/cr2zm2px70q9hrgj23zmw7_r0000gn/T/ipykernel_25520/3674209955.py:1: DtypeWarning: Columns (20,39,40,41,43,47,48,49,52,54,57,58,60,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicles = pd.read_csv("data/Traffic_Crashes_Vehicles.csv")
/var/folders/4f/cr2zm2px70q9hrgj23zmw7_r0000gn/T/ipykernel_25520/3674209955.py:2: DtypeWarning: Columns (19,23,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people = pd.read_csv("data/Traffic_Crashes_People.csv")


## Merge Dataframes

First we sample 300,000 crash reports, then we merge the vehicle and people involved. 

In [96]:
#Random sample of vehicles 
df_accidents = df_vehicles.sample(n=300000, random_state=42)

In [107]:
df_accidents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 1459795 to 1284668
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   PERSON_ID              300000 non-null  object 
 1   PERSON_TYPE            300000 non-null  object 
 2   CRASH_RECORD_ID        300000 non-null  object 
 3   VEHICLE_ID             293876 non-null  float64
 4   CRASH_DATE             300000 non-null  object 
 5   SEAT_NO                60945 non-null   float64
 6   CITY                   218536 non-null  object 
 7   STATE                  221782 non-null  object 
 8   ZIPCODE                200973 non-null  object 
 9   SEX                    294986 non-null  object 
 10  AGE                    212894 non-null  float64
 11  DRIVERS_LICENSE_STATE  175575 non-null  object 
 12  DRIVERS_LICENSE_CLASS  146281 non-null  object 
 13  SAFETY_EQUIPMENT       299164 non-null  object 
 14  AIRBAG_DEPLOYED        294146 non-

In [102]:
#Then we can add the crash report data 
df_accidents = df_accidents.merge(df_crashes, how="inner", on=["CRASH_RECORD_ID", "CRASH_DATE"])

In [133]:
df_accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324360 entries, 0 to 324359
Columns: 236 entries, Unnamed: 0 to CELL_PHONE_USE
dtypes: float64(46), int64(21), object(169)
memory usage: 584.0+ MB


In [113]:
#every person and vehicle should have a match for the record ID, vehicle ID, and crash date
#This makes the dataframe larger as multiple people are involved in each accident 
df_accidents = df_accidents.merge(df_people, how="inner", on=["CRASH_RECORD_ID", "VEHICLE_ID", "CRASH_DATE"])

In [114]:
df_accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324360 entries, 0 to 324359
Columns: 235 entries, CRASH_UNIT_ID to CELL_PHONE_USE
dtypes: float64(45), int64(20), object(170)
memory usage: 581.5+ MB


The dataframe gets larger when merging in the people, as there are sometimes more than one person involved in an accident.  

# THIS IS FOR WORKING PURPOSES. 

# USE THIS TO IMPORT DF WITHOUT RUNNING THE CELLS ABOVE.

## Export Sample Data for People and Crashes

In [129]:
df_people_sample = df_people[df_accidents.isin(df_people['CRASH_RECORD_ID'])]
df_crashes_sample = df_crashes[df_crashes.isin(df_accidents['CRASH_RECORD_ID'])]

In [130]:
df_people_sample.head()

,Unnamed: 0.1,Unnamed: 0,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
df_people_sample.to_csv('data/People_Sample.csv')
df_people_sample.to_csv('data/Crashes_Sample.csv')

In [126]:
df_people = pd.read_csv("data/People_Sample.csv")
df_crashes = pd.read_csv("data/Crashes_Sample.csv")

## Export Final

In [115]:
df_accidents.to_csv('data/Final_Data.csv')

In [116]:
df_accidents = pd.read_csv("data/Final_Data.csv")

/var/folders/4f/cr2zm2px70q9hrgj23zmw7_r0000gn/T/ipykernel_25520/2274169522.py:1: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,55,58,59,61,71,102,148,194,215,235) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accidents = pd.read_csv("data/Final_Data.csv")


# ^^^^^ DELETE BEFORE SUBMISSION

In [117]:
count = 0
for column in df_accidents.iloc[0]:
    print(df_accidents.columns[count], ":", column, ",", df_accidents.iloc[1].iloc[count])
    count += 1

Unnamed: 0 : 0 , 1
CRASH_UNIT_ID : 1344100 , 748747
CRASH_RECORD_ID : 3d1a5bb0d4c56b1671566a15ce7181e94da4ab874764fe39bd113d58f4d0a1592d290ef57916d1c02979aedcce013a08db670d24550a60639ee0ed232c6da4d5 , b1cee0206d4a3d17e14599074fe0d01f73c16f4fdfa5592f3d966669d12e5d5b7f49f122b2a695c657ee67a29006ac21da78f4567ac9801f1d196a4958a8ca00
CRASH_DATE : 05/24/2022 08:23:00 PM , 09/28/2019 01:45:00 PM
UNIT_NO : 1 , 2
UNIT_TYPE : DRIVER , DRIVER
NUM_PASSENGERS : nan , nan
VEHICLE_ID : 1276996.0 , 711349.0
CMRC_VEH_I : nan , nan
MAKE : TOYOTA , VOLKSWAGEN
MODEL : CAMRY , JETTA
LIC_PLATE_STATE : IL , IL
VEHICLE_YEAR : 2018.0 , 2012.0
VEHICLE_DEFECT : UNKNOWN , UNKNOWN
VEHICLE_TYPE : PASSENGER , PASSENGER
VEHICLE_USE : PERSONAL , PERSONAL
TRAVEL_DIRECTION : W , S
MANEUVER : TURNING LEFT , STRAIGHT AHEAD
TOWED_I : nan , nan
FIRE_I : nan , nan
OCCUPANT_CNT : 1.0 , 1.0
EXCEED_SPEED_LIMIT_I : nan , nan
TOWED_BY : nan , nan
TOWED_TO : nan , nan
AREA_00_I : Y , nan
AREA_01_I : nan , nan
AREA_02_I : nan , nan


For Group Project: Create CSVs of the truncated csvs

## Drop Unused Columns

Use a Decsicion Tree model to determine importaint categories. 

### DT

In [118]:
columns_to_drop = [
    'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 
    'TRAILER1_LENGTH','TRAILER2_LENGTH',
    'HAZMAT_OUT_OF_SERVICE_I', 'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS', 'FIRE_I', 'TOWED_I', 'TOWED_BY', 'TOWED_TO', 
    'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'GVWR', 
    'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY',
    'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO', 'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO', 
    'MCS_REPORT_I', 'MCS_REPORT_NO','HAZMAT_VIO_CAUSE_CRASH_I', 'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO',
    'AREA_00_I', 'AREA_01_I', 'AREA_02_I',        
    'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I',
    'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 'AREA_10_I',
    'AREA_11_I', 'AREA_12_I', 'AREA_99_I',
    'CMRC_VEH_I', 'LOCATION', 'STREET_DIRECTION', 'DATE_POLICE_NOTIFIED',
    'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE', 'LANE_CNT',
    'ALIGNMENT', 'INTERSECTION_RELATED_I', 'BEAT_OF_OCCURRENCE', 
    'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I',
    'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I',
    'STREET_NO', 'ZIPCODE', 'DRIVERS_LICENSE_CLASS',
    'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION',
    'DRIVER_VISION', 'STATE', 'CITY', 'MANEUVER',

]

df_accidents.drop(columns=columns_to_drop)

,Unnamed: 0,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,MAKE,MODEL,...,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,0,1344100,3d1a5bb0d4c56b1671566a15ce7181e94da4ab874764fe...,05/24/2022 08:23:00 PM,1,DRIVER,NaN,1276996.0,TOYOTA,CAMRY,...,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,1,748747,b1cee0206d4a3d17e14599074fe0d01f73c16f4fdfa559...,09/28/2019 01:45:00 PM,2,DRIVER,NaN,711349.0,VOLKSWAGEN,JETTA,...,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,2,1304152,62a78b0281ac282ceb89d3132ac5ba7e5e3ccb3bac5347...,03/24/2022 05:29:00 AM,3,DRIVER,NaN,1238532.0,GREAT DANE TRAILERS INC.,GREAT DANE TRAILERS INC.,...,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,3,510444,d6ff5e6a5d94ab5e99ad3fbf64487cb06973ca089279da...,10/29/2018 08:43:00 AM,1,DRIVER,NaN,487775.0,HONDA,ACCORD,...,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,4,1397785,9a2dce49739ffd63f3ea7d5bef721812eb7c2cb231c1b0...,08/12/2022 08:25:00 PM,3,DRIVER,1.0,1328219.0,MERCEDES-BENZ,OTHER (EXPLAIN IN NARRATIVE),...,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324355,324355,981844,856a1c0b8f4cb6b11d5f2fbd17caff3be1b539b3ff9bd0...,10/23/2020 11:38:00 AM,2,DRIVER,NaN,930909.0,FORD,TRANSIT CONNECT,...,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
324356,324356,627295,98892cddfb51fd9bbf244f8c00f39dd5e72fdb6e141341...,04/17/2019 04:14:00 PM,1,DRIVER,2.0,597536.0,FORD,Explorer,...,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324357,324357,627295,98892cddfb51fd9bbf244f8c00f39dd5e72fdb6e141341...,04/17/2019 04:14:00 PM,1,DRIVER,2.0,597536.0,FORD,Explorer,...,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324358,324358,627295,98892cddfb51fd9bbf244f8c00f39dd5e72fdb6e141341...,04/17/2019 04:14:00 PM,1,DRIVER,2.0,597536.0,FORD,Explorer,...,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NORMAL,NaN,NaN,NaN,TEST REFUSED,NaN,NaN


## Combine Columns into Categories 

In [82]:
#py

## Train Test Split

# Model 1

# Model 2

# Conclusions 